# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 67 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.01946


extracting tarball to tmp_2302.01946...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.02005


extracting tarball to tmp_2302.02005...

...

extracting tarball to tmp_2302.02915...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.02927


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2302.02915/Egal2023a.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2302.02927...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.02429-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.02429) | **Life beyond 30: probing the -20<M_UV<-17 luminosity function at 8<z<13  with the NIRCam parallel field of the MIRI Deep Survey**  |
|| Pablo G. Pérez-González, et al. -- incl., <mark>Danial Langeroodi</mark>, <mark>Almudena Alonso-Herrero</mark>, <mark>Fabian Walter</mark>, <mark>Sarah E. I. Bosman</mark> |
|*Appeared on*| *2023-02-07*|
|*Comments*| *Submitted to ApJL, in memoriam Hans Ulrik Noorgaard-Nielsen and Olivier Le F\`evre*|
|**Abstract**| We present the ultraviolet luminosity function and an estimate of the cosmic star formation rate density at $8<z<13$ derived from deep NIRCam observations taken in parallel with the MIRI Deep Survey (MDS) of the Hubble Ultra Deep Field (HUDF), NIRCam covering the parallel field 2 (HUDF-P2). Our deep (40~hours) NIRCam observations reach a $F277W$ magnitude of 32 at the $5\sigma$ level, more than 2 magnitudes deeper than JWST public datasets already analyzed to find high redshift galaxies. We select a sample of 45 $z>8$ galaxy candidates based on their dropout nature in the $F115W$ and/or $F150W$ filters, a high probability for their photometric redshifts, estimated with three different codes, being at $z>8$, good fits based on $\chi^2$ calculations, and predominant solutions compared to $z<8$ alternatives. We find mild evolution in the luminosity function from $z\sim13$ to $z\sim8$, i.e., only a small increase in the average number density of $\sim$0.2~dex, while the faint-end slope and absolute magnitude of the knee remain approximately constant, with values $\alpha=-2.3\pm0.2$ and $M^*=-20.8\pm0.2$~mag. Comparing our results with the predictions of a wide range of state-of-the-art galaxy evolution models, we find two main results: (1) a slower increase with time in the cosmic star formation rate density compared to a steeper rise predicted by models; (2) nearly a factor of 10 higher star formation activity concentrated in scales around 2~kpc in galaxies with stellar masses $\sim10^8$~M$_\odot$ during the first 350~Myr of the Universe ($z\sim12$), with models matching better the observations $\sim$150~Myr later, by $z\sim9$. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.02611-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.02611) | **Robust Data-driven Metallicities for 120 Million Stars from Gaia XP  Spectra**  |
|| <mark>Rene Andrae</mark>, <mark>Hans-Walter Rix</mark>, Vedant Chandra |
|*Appeared on*| *2023-02-07*|
|*Comments*| *18 pages, 17 figures, submitted to ApJS*|
|**Abstract**| We derive and publish data-driven estimates of stellar metallicities [M/H] for ~120 million stars with low-resolution XP spectra published in Gaia DR3. The [M/H] values, along with Teff and logg, are derived using the XGBoost algorithm, trained on stellar parameters from APOGEE, augmented by a set of very metal-poor stars. XGBoost draws on a number of data features: the full set of XP spectral coefficients, narrowband fluxes derived from XP spectra, and broadband magnitudes. In particular, we include AllWISE magnitudes, as they reduce the degeneracy of Teff and dust reddening. We also include the parallax as a data feature, which helps constrain logg and [M/H]. The resulting mean stellar parameter precision is 0.1dex in [M/H], 50K in Teff, and 0.08dex in logg. This all-sky [M/H] sample is two orders of magnitude larger than published samples of comparable fidelity across -3<[M/H]<+0.5. Additionally, we provide a catalog of over 13 million bright (G<16) red giants whose [M/H] are vetted to be precise and pure. We present all-sky maps of the Milky Way in different [M/H] regimes that illustrate the purity of the dataset, and demonstrate the power of this unprecedented sample to reveal the Milky Way's structure from its heart to its disk. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.02890-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.02890) | **Evidence of an age gradient along the line of sight in the nuclear  stellar disc of the Milky Way**  |
|| <mark>F. Nogueras-Lara</mark>, et al. |
|*Appeared on*| *2023-02-07*|
|*Comments*| *Submitted to A&A*|
|**Abstract**| The nuclear stellar disc (NSD) is a flat dense stellar structure at the heart of the Milky Way. Recent work shows that analogous structures are common in the nuclei of external spiral galaxies, where there is evidence of an age gradient that indicates that they form inside-out. However, the characterisation of the age of the NSD stellar population along the line of sight is still missing due to its extreme source crowding and the high interstellar extinction towards the Galactic centre. We aim to characterise the age of the stellar population at different average Galactocentric NSD radii to investigate for the first time the presence of an age gradient along the line of sight. We selected two groups of stars at different NSD radii via their different extinction and proper motion distribution. We analysed their stellar population by fitting their de-reddened $K_s$ luminosity functions with a linear combination of theoretical models. We find significant differences in the stellar population at different NSD radii, indicating the presence of an age gradient along the line of sight. Our sample from the closest edge of the NSD contains a significant fraction ($\sim40$ % of its total stellar mass) of intermediate-age stars (2-7 Gyr), that is not present in the sample from stars deeper inside the NSD, in which $\sim90 %$ of the stellar mass is older than 7 Gyr. Our results suggest that the NSD age distribution is similar as the one found in external galaxies and imply that bar-driven processes observed in external galaxies are similarly at play in the Milky Way. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.02897-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.02897) | **Understanding and predicting cadence effects in the characterization of  exoplanet transits**  |
|| <mark>Julio Hernandez Camero</mark>, Cynthia S. K. Ho, Vincent Van Eylen |
|*Appeared on*| *2023-02-07*|
|*Comments*| *15 pages, 9 figures. Accepted for publication in MNRAS*|
|**Abstract**| We investigate the effect of observing cadence on the precision of radius ratio values obtained from transit light curves by performing uniform Markov Chain Monte Carlo fits of 46 exoplanets observed by the Transiting Exoplanet Survey Satellite (TESS) in multiple cadences. We find median improvements of almost 50% when comparing fits to 20s and 120s cadence light curves to 1800s cadence light curves, and of 37% when comparing 600s cadence to 1800s cadence. Such improvements in radius precision are important, for example, to precisely constrain the properties of the radius valley or to characterize exoplanet atmospheres. We also implement a numerical Information Analysis to predict the precision of parameter estimates for different observing cadences. We tested this analysis on our sample and found it reliably predicts the effect of shortening observing cadence with errors in the predicted % precision of <0,5% for most cases. We apply this method to 157 TESS object of interest that have only been observed with 1800s cadence to predict the precision improvement that could be obtained by reobservations with shorter cadences and provide the full table of expected improvements. We report the 10 planet candidates that would benefit the most from reobservations at short cadence. Our implementation of the Information Analysis for the prediction of the precision of exoplanet parameters, Prediction of Exoplanet Precisions using Information in Transit Analysis (PEPITA) is made publicly available. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.02915-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.02915) | **Modelling the 2022 τ-Herculid outburst**  |
|| Auriane Egal, Paul A. Wiegert, <mark>Peter G. Brown</mark>, Denis Vida |
|*Appeared on*| *2023-02-07*|
|*Comments*| *Accepted for publication in the Astrophysical Journal (Feb. 3, 2023)*|
|**Abstract**| The $\tau$-Herculids (IAU shower number #61 TAH) is a minor meteor shower associated with comet 73P/Schwassmann-Wachmann 3, a Jupiter-Family comet that disintegrated into several fragments in 1995. As a consequence of the nucleus break-up, possible increased meteor rates were predicted for 2022. On May 30-31, observation networks around the world reported two distinct peaks of TAH activity, around solar longitudes 69.02{\deg} and 69.42{\deg}. This work examines the encounter conditions of the Earth with meteoroids ejected from 73P during the splitting event and on previous perihelion passages. Numerical simulations suggest that the main peak observed in 2022 was caused by meteoroids ejected from the splitting nucleus with four times the typical cometary gas expansion speed. High-resolution measurements performed with the Canadian Automated Meteor Observatory indicate that these meteoroids are fragile, with estimated bulk densities of 250 kg/m$^3$. In contrast with the main peak, the first TAH activity peak in 2022 is best modelled with trails ejected prior to 1960. We find that ordinary cometary activity could have produced other TAH apparitions observed in the past, including in 1930 and 2017. The extension of our model to future years predicts significant returns of the shower in 2033 and 2049. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.02927-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.02927) | **Handling Background in IXPE polarimetric data**  |
|| Alessandro Di Marco, et al. -- incl., <mark>Fei Xie</mark> |
|*Appeared on*| *2023-02-07*|
|*Comments*| **|
|**Abstract**| IXPE (Imaging X-ray Polarimetry Explorer) is a Small Explorer mission by NASA and ASI, launched on December 9$^{th}$ 2021, dedicated to investigating X-ray polarimetry allowing angular-, time- and energy-resolved observations in the 2--8 keV energy band. IXPE is in Science Observation phase since January 2022; it comprises of three identical telescopes with grazing-incidence mirrors, each one having in the focal plane a Gas Pixel Detector (GPD). In this paper, we present a possible guideline to obtain an optimal background selection in polarimetric analysis, and a rejection strategy to remove instrumental background. This work is based on the analysis of IXPE observations, aiming to improve as much as possible the polarimetric sensitivity. In particular, the developed strategies have been applied ``as a case study'' to the IXPE observation of the 4U 0142+61 magnetar. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.02802-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.02802) | **A Systematic Search for Short-period Close White Dwarf Binary Candidates  Based on Gaia EDR3 Catalog and Zwicky Transient Facility Data**  |
|| <mark>Liangliang Ren</mark>, et al. -- incl., <mark>Chengyuan Li</mark> |
|*Appeared on*| *2023-02-07*|
|*Comments*| *40 pages, 20 figures*|
|**Abstract**| Galactic short-period close white dwarf binaries (CWDBs) are important objects for space-borne gravitational-wave (GW) detectors in the millihertz frequency bands. Due to the intrinsically low luminosity, only about 25 identified CWDBs are detectable by the Laser Interferometer Space Antenna (LISA), which are also known as verification binaries (VBs). The Gaia Early Data Release 3 (EDR3) provids a catalog containing a large number of CWDB candidates, which also includes parallax and photometry measurements. We crossmatch the Gaia EDR3 and Zwicky Transient Facility public data release 8, and apply period-finding algorithms to obtain a sample of periodic variables. The phase-folded light curves are inspected, and finally we obtain a binary sample containing 429 CWDB candidates. We further classify the samples into eclipsing binaries (including 58 HW Vir-type binaries, 65 EA-type binaries, 56 EB-type binaries, and 41 EW-type binaries) and ellipsoidal variations (209 ELL-type binaries). We discovered four ultrashort period binary candidates with unique light-curve shapes. We estimate the GW amplitude of all of our binary candidates, and calculate the corresponding signal-to-noise ratio (S/N) for TianQin and LISA. We find two (six) potential GW candidates with S/Ns greater than 5 in the nominal mission time of TianQin (LISA), which increases the total number of candidate VBs for TianQin (LISA) to 18 (31). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.02359-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.02359) | **A Real-Time Re-Scheduling Algorithm for Spacecraft Instrument Operations  Optimization**  |
|| J. Pergoli, et al. -- incl., <mark>P. Di Lizia</mark> |
|*Appeared on*| *2023-02-07*|
|*Comments*| **|
|**Abstract**| The operation-planning of satellites, aimed at introducing a certain level of supervised automation during the execution of the operations, poses a great challenge to both designers and operators. From one side, the routine operations for an Earth Observation mission are predictable and typically repeatable; both these aspects are suitable for computerisation. On the other hand, not every non-nominal scenario can be anticipated and correctly formulated in terms of operations. Dealing with contingency presents risks which need to be addressed as early as possible, hopefully already during the operations preparation. It is also possible, however, to intervene at a later operational stage of the mission, optimising the tools already in use to support the operations execution. In this paper, having in mind the idea to improve existing processes in place at EUMETSAT, we present an algorithm able to reschedule the spacecraft's activities in case of anomaly. The main goal is to support the decision-making process while overcoming contingencies both avoiding overloading the spacecraft and planning engineers and ensuring the continuity of the mission, in particular giving the highest priority to the onboard computer memory size and data quality. We tested the method with the data of Sentinel-6, which carries the altimeter POSEIDON-4 operated by EUMETSAT, and the results are hereby presented. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.02429.md
    + _build/html/tmp_2302.02429/./fig_density_v20230121.png
    + _build/html/tmp_2302.02429/./plot_lf_v20230121.png
    + _build/html/tmp_2302.02429/./fig_mag_reshift_v20230121.png
exported in  _build/html/2302.02611.md
    + _build/html/tmp_2302.02611/./internal-x-validation-XGBoost.png
    + _build/html/tmp_2302.02611/./Praesepe-MH-vs-colour.png
    + _build/html/tmp_2302.02611/./skymap-mh-slice-A.png
    + _build/html/tmp_2302.02611/./skymap-mh-slice-B.png
    + _build/html/tmp_2302.02611/./skymap-mh-slice-C.png
    + _build/html/tmp_2302.02611/./skymap-vetted-mh-slice-A.png
    + _build/html/tmp_2302.02611/./skymap-vetted-mh-slice-B.png
    + _build/html/tmp_2302.02611/./skymap-vetted-mh-slice-C.png
exported in  _build/html/2302.02890.md
    + _build/html/tmp_2302.02890/./CMD.png
    + _build/html/tmp_2302.02890/./res_fit.png
    + _build/html/tmp_2302.02890/./KLFs.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\lucacomment}[1]{{\color{magenta} [{#1}]}}$
$\newcommand{\luca}[1]{\lucacomment{{\bf #1}}}$
$\newcommand{\mariannacomment}[1]{{\color{blue} [{#1}]}}$
$\newcommand{\MA}[1]{\mariannacomment{{\bf #1}}}$
$\newcommand{\danial}[1]{{\color{teal}{#1}}}$
$\newcommand{\johncomment}[1]{{\color{green} [{#1}]}}$
$\newcommand{\john}[1]{\johncomment{{\bf #1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\lucacomment$}[1]{{\color{magenta} [{#1}]}}$
$\newcommand{$\luca$}[1]{$\lucacomment${{\bf #1}}}$
$\newcommand{$\mariannacomment$}[1]{{\color{blue} [{#1}]}}$
$\newcommand{$\MA$}[1]{$\mariannacomment${{\bf #1}}}$
$\newcommand{$\danial$}[1]{{\color{teal}{#1}}}$
$\newcommand{$\johncomment$}[1]{{\color{green} [{#1}]}}$
$\newcommand{$\john$}[1]{$\johncomment${{\bf #1}}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

#  with the NIRCam parallel field of the MIRI Deep Survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.02429-b31b1b.svg)](https://arxiv.org/abs/2302.02429)<mark>Appeared on: 2023-02-07</mark> - _Submitted to ApJL, in memoriam Hans Ulrik Noorgaard-Nielsen and Olivier Le F\`evre_

</div>
<div id="authors">

Pablo G. P\'erez-Gonz\'alez, et al. -- incl., <mark>Danial Langeroodi</mark>, <mark>Almudena Alonso-Herrero</mark>, <mark>Fabian Walter</mark>, <mark>Sarah E.~I.~Bosman</mark>

</div>
<div id="abstract">

**Abstract:** We present the ultraviolet luminosity function and an estimate of the cosmic star formation rate density at$8<z<13$derived from deep NIRCam observations taken in parallel with the MIRI Deep Survey (MDS) of the Hubble Ultra Deep Field (HUDF), NIRCam covering the parallel field 2 (HUDF-P2). Our deep (40 hours) NIRCam observations reach a$F277W$magnitude of 32 at the$5\sigma$level, more than 2 magnitudes deeper than JWST public datasets already analyzed to find high redshift galaxies. We select a sample of 45$z>8$galaxy candidates based on their dropout nature in the$F115W$and/or$F150W$filters, a high probability for their photometric redshifts, estimated with three different codes, being at$z>8$, good fits based on$\chi^2$calculations, and predominant solutions compared to$z<8$alternatives. We find mild evolution in the luminosity function from$z\sim13$to$z\sim8$, i.e., only a small increase in the average number density of$\sim$0.2 dex, while the faint-end slope and absolute magnitude of the knee remain approximately constant, with values$\alpha=-2.3\pm0.2$and$M^*=-20.8\pm0.2$mag. Comparing our results with the predictions of a wide range of state-of-the-art galaxy evolution models, we find two main results: (1) a slower increase with time in the cosmic star formation rate density compared to a steeper rise predicted by models; (2) nearly a factor of 10 higher star formation activity concentrated in scales around 2 kpc in galaxies with stellar masses$\sim10^8$M$_\odot$during the first 350 Myr of the Universe ($z\sim12$), with models matching better the observations$\sim$150 Myr later, by$z\sim9$.

</div>

<div id="div_fig1">

<img src="tmp_2302.02429/./fig_density_v20230121.png" alt="Fig4" width="100%"/>

**Figure 4. -** \label{fig:density}Evolution of the UV luminosity density, transformed into SFR density on the right vertical axis (assuming a \citealt{2003PASP..115..763C} IMF). Our results are plotted with a red shaded region. Estimates from the literature are plotted with gray symbols \citep{2018ApJ...855..105O,2022arXiv221206683B,2022arXiv221102607B,2023MNRAS.518.6011D,2022arXiv220801612H}. We also plot predictions from several galaxy formation simulations, listed in the following lines. The Illustris-1, {\sc TNG50, TNG100, TNG300} results are plotted with blue continuous, dashed, dotted, and dash-dotted lines, respectively, all  referring to SFRs measured in 0.5$\arcsec$  diameter apertures for $\mathrm{M\!_\star}>10^{7}$ M$_\odot$ galaxies, except for {\sc TNG300}, cut at $\mathrm{M\!_\star}>10^{8}$ M$_\odot$). The thick dotted line refers to {\sc TNG100} results using whole galaxy measurements \citealt{2014MNRAS.444.1518V, 2015A&C....13...12N, 2018MNRAS.475..676S, 2018MNRAS.477.1206N, 2018MNRAS.480.5113M, 2018MNRAS.475..624N, 2018MNRAS.475..648P, 2019MNRAS.490.3196P,2019MNRAS.490.3234N}). The {\sc EAGLE} simulations \citep{2015MNRAS.446..521S,2015MNRAS.450.1937C} are plotted in green for galaxy samples cut in stellar masses $\mathrm{M\!_\star}>10^{6,7,8}$ M$_\odot$ in continuous, dotted, dashed lines. The {\sc simba} predictions are shown with a brown continuous line for all galaxies, dashed line for $F277W<31$ mag sources \citep{2019MNRAS.486.2827D}. The {\sc thesan-1} and {\sc thesan-sdao-2} simulations \citep{2022MNRAS.511.4005K} are shown in magenta with continuous and dotted lines, respectively (see text for differences). The {\sc FLARES} predictions are shown in cyan \citep{2021MNRAS.500.2127L,2021MNRAS.501.3289V}. The  UniverseMachine semi-empirical model \citep{2020MNRAS.499.5702B} is shown in orange with the same mass cuts and line styles mentioned for {\sc EAGLE}. Finally, the models in \citet{2022arXiv221103620M} are shown in yellow. (*fig:density*)

</div>
<div id="div_fig2">

<img src="tmp_2302.02429/./plot_lf_v20230121.png" alt="Fig3" width="100%"/>

**Figure 3. -** \label{fig:lf}From left to right, UV luminosity functions at $z\sim9$, $z\sim11$ and $z\sim12$. We plot our results with red hexagons (uncertainties smaller than data points in some cases, see Table \ref{tab:lf}), and compare with literature estimations from \citet{2023MNRAS.518.6011D,2022arXiv220709434N,2022arXiv221105792F,2022arXiv221206683B},  \citet{2022arXiv220801612H}, and \citet{2022arXiv221206666C}. We fit all points (ours and those in the literature) to a  \citet{1976ApJ...203..297S} function, which is plotted with red continuous lines in each panel, with the fits for other redshifts shown with dashed lines. (*fig:lf*)

</div>
<div id="div_fig3">

<img src="tmp_2302.02429/./fig_mag_reshift_v20230121.png" alt="Fig2" width="100%"/>

**Figure 2. -** \label{fig:z-mag}Magnitude in the NIRCam $F277W$ filter vs$.$ photometric redshift for $z>8$ galaxies. General galaxy samples extracted from the catalogs released by G. Brammer for the SMACS0723, GLASS, and CEERS datasets gathered in 2022 are plotted with blue small dots  to illustrate the magnitude limits of the surveys and the number of high-redshift galaxies in unsupervised and unvetted photometric catalogs. Our parent galaxy sample is plotted with red small dots, and the final sample of $z>8$ galaxy candidates with red hexagons. We depict $1\sigma$ uncertainties plotted in both axes and the data point referring to the zPDF peak redshift. We also plot the photometric high-redshift galaxy candidates of \citet{2023MNRAS.518.6011D}, \citet{2022arXiv220709434N}, \citet{2022arXiv221206683B}, and \citet{2022arXiv221105792F} with large black circles (samples not providing $F277W$ magnitudes are not shown, e.g., \citet{2022arXiv220801612H}, and we do not remove repeated sources with different measurements), as well as spectroscopically confirmed galaxies from JADES \citep{2022arXiv221204568C,2022arXiv221204480R} and lensing clusters \citep[][not corrected for magnification]{2023MNRAS.518L..45C,2022arXiv221015699W,2022arXiv221202491L,2022arXiv221015639R} in green. The top panel shows the histogram of redshifts for these samples of high-redshift galaxy candidates. The right panel shows the magnitude distribution of the different datasets mentioned before, with the three 2022 surveys plotted with  transparency. Our deeper data (peaking around 2 magnitudes fainter than previous surveys) probe the 29 to 31 mag regime, providing candidates at the previously unexplored faint-end of the luminosity functions. (*fig:z-mag*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Robust Data-driven Metallicities for 120 Million Stars from Gaia XP Spectra

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.02611-b31b1b.svg)](https://arxiv.org/abs/2302.02611)<mark>Appeared on: 2023-02-07</mark> - _18 pages, 17 figures, submitted to ApJS_

</div>
<div id="authors">

<mark>Ren\'e Andrae</mark>, <mark>Hans-Walter Rix</mark>, Vedant Chandra

</div>
<div id="abstract">

**Abstract:** We derive and publish data-driven estimates of stellar metallicities\MHfor$\gtrsim 120$million stars with low-resolution XP spectra published in Gaia DR3.The\MHvalues, along with\Teffand\logg,are derived using the XGBoost algorithm, trained on stellar parameters from APOGEE, augmented by a set of very metal-poor stars.XGBoost draws on a number of data features: the full set of XP spectral coefficients, narrowband fluxes derived from XP spectra, and broadband magnitudes.  In particular, we include AllWISE magnitudes, as they reduce the degeneracy of\Teffand dust reddening.  We also include the parallax as a data feature, which helps constrain\loggand\MH. The resulting mean stellar parameter precision is 0.1 dex in\MH, 50 K in\Teff, and 0.08 dex in\logg.This all-sky\MHsample  is two orders of magnitude larger than published samples of comparable fidelity across$-3\lesssim \MH \lesssim +0.5$.Additionally, we provide a catalog of over 13 million bright ($G<16$) red giants whose\MHare vetted to be precise and pure.We present all-sky maps of the Milky Way in different\MHregimes that illustrate the purity of the dataset, and demonstrate the power of this unprecedented sample to reveal the Milky Way's structure from its heart to its disk.

</div>

<div id="div_fig1">

<img src="tmp_2302.02611/./internal-x-validation-XGBoost.png" alt="Fig14" width="100%"/>

**Figure 14. -** Cross-validation of the XGBoost parameters derived from the XP spectra with SDSS-APOGEE DR17. The plots show the results of the twenty-fold cross-validation of the 5\% portions of the training sample, withheld in the training. Rows from top to bottom show \MH residuals, \Teff residuals and \logg residuals. Columns from left to right show residuals vs. APOGEE's \MH, APOGEE's \Teff, APOGEE's \logg and Gaia's apparent $G_\textrm{BP}$ magnitude. The numbers in the top right corners quote the median absolute difference (MedAD) and the root mean square difference (RMSD). The density map is logarithmic. These plots illustrate the remarkable precision of the approach: 0.10 dex in \MH, 50K in \Teff , and 0.08 dex in \logg . Note that these variances still include all the uncertainties in the APOGEE estimates. (*fig:internal-x-validation-XGBoost*)

</div>
<div id="div_fig2">

<img src="tmp_2302.02611/./Praesepe-MH-vs-colour.png" alt="Fig7" width="100%"/>

**Figure 7. -** Validation of the \MH estimates in the main sequence, using the Praesepe cluster. The cluster's color-magnitude diagram of all 640 members with \MH is illustrated in panel (a). Their \MH estimates are shown as a function of color in panel (b). The horizontal dashed line indicates the metallicity of 0.16 quoted by \citet{2018A&A...618A..93C}. For $0.5<G_\textrm{BP}-G_\textrm{RP}<1.5$ the metallicity agreement is excellent, whereas for $1.5<G_\textrm{BP}-G_\textrm{RP}<2.5$ they are systematically too low by 0.15 dex. Outside of these color-ranges the agreement is poor. We attribute the offsets and the poor estimates to possible systematics and poor sampling of the CMD space in the training sample. The \MH estimates for main-sequence stars with colors outside $0.5<G_\textrm{BP}-G_\textrm{RP}<2.5$ are manifestly unreliable. (*fig:Praesepe-MH-vs-colour*)

</div>
<div id="div_fig3">

<img src="tmp_2302.02611/./skymap-mh-slice-A.png" alt="Fig16.1" width="16%"/><img src="tmp_2302.02611/./skymap-mh-slice-B.png" alt="Fig16.2" width="16%"/><img src="tmp_2302.02611/./skymap-mh-slice-C.png" alt="Fig16.3" width="16%"/><img src="tmp_2302.02611/./skymap-vetted-mh-slice-A.png" alt="Fig16.4" width="16%"/><img src="tmp_2302.02611/./skymap-vetted-mh-slice-B.png" alt="Fig16.5" width="16%"/><img src="tmp_2302.02611/./skymap-vetted-mh-slice-C.png" alt="Fig16.6" width="16%"/>

**Figure 16. -** Skymaps showing the logarithmic number density of all \emph{unfiltered} stars with $-3<\MH<-1.2$(top panel), $-0.9<\MH<-0.6$(middle panel) and $-0.3<\MH<0.1$(bottom panel). These illustrate two important issues: the incompleteness of XP spectra in the two sickle-shaped regions at high latitude. And a significant contamination of the metal-poor bin in the unfiltered sample, which manifests itself as a thin disk near the Galactic plane; these are presumably hotter and highly reddened stars with weaker metal lines that are not recognized as such by our algorithm, as it lacks good training sets in this CMD regime.Skymaps showing the logarithmic number density of \emph{vetted} RGB stars for $-3<\MH<-1.2$(top panel), $-0.9<\MH<-0.6$(middle panel) and $-0.3<\MH<0.1$(bottom panel). This vetted sample (see Fig. \ref{fig:clean-RGB-cuts}) is restricted to red giants with good S/N ($G<16$), \Teff cuts that eliminate contaminants in the low-metallicity subsample and significant parallax (which explains the "disapperance" of the Magellanic Clouds compared to Fig. \ref{fig:mono-abundance-all-sky}. The top panel qualitatively illustrates how clean the metal-poor subsample is: it prominently shows the \emph{Poor Old Heart of the Galaxy}\citep{Rix2022}, without any traces of spurious sample members in the disk that is so dominant in the high-metallicity subsample (bottom panel). (*fig:mono-abundance-all-sky*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Evidence of an age gradient along the line of sight in the nuclear stellar disc of the Milky Way

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.02890-b31b1b.svg)](https://arxiv.org/abs/2302.02890)<mark>Appeared on: 2023-02-07</mark> - _Submitted to A&A_

</div>
<div id="authors">

<mark>F. Nogueras-Lara</mark>, et al.

</div>
<div id="abstract">

**Abstract:** The nuclear stellar disc (NSD) is a flat dense stellar structure at the heart of the Milky Way. Recent work shows that analogous structures are common in the nuclei of external spiral galaxies, where there is evidence of an age gradient that indicates that they form inside-out. However, the characterisation of the age of the NSD stellar population along the line of sight is still missing due to its extreme source crowding and the high interstellar extinction towards the Galactic centre.We aim to characterise the age of the stellar population at different average Galactocentric NSD radii to investigate for the first time the presence of an age gradient along the line of sight.We selected two groups of stars at different NSD radii via their different extinction and proper motion distribution. We analysed their stellar population by fitting their de-reddened$K_s$luminosity functions with a linear combination of theoretical models.We find significant differences in the stellar population at different NSD radii, indicating the presence of an age gradient along the line of sight. Our sample from the closest edge of the NSD contains a significant fraction ($\sim40$\%of its total stellar mass) of intermediate-age stars (2-7 Gyr), that is not present in the sample from stars deeper inside the NSD, in which$\sim90 \%$of the stellar mass is older than 7 Gyr. Our results suggest that the NSD age distribution is similar as the one found in external galaxies and  imply that bar-driven processes observed in external galaxies are similarly at play in the Milky Way.

</div>

<div id="div_fig1">

<img src="tmp_2302.02890/./CMD.png" alt="Fig1" width="100%"/>

**Figure 1. -** CMD $K_s$ versus $H-K_s$. The blue and salmon coloured regions indicate the two target groups of stars with different reddening dominated on average by stars from the closest edge of the NSD (NSD outer region) and stars deeper inside the NSD  (NSD inner region), respectively. The dashed boxes show the reference stars used to build the extinction maps for each of the stellar groups analysed (see Sect, \ref{extinct}). The black arrow indicates the reddening vector. (*CMD*)

</div>
<div id="div_fig2">

<img src="tmp_2302.02890/./res_fit.png" alt="Fig6" width="100%"/>

**Figure 6. -** Stellar populations present in the NSD outer region (left panel), inner region (central panel), and all the NSD stars in the sample (right panel). The numbers indicate the percentage of mass due to a given age bin and its standard deviation. (*SFH*)

</div>
<div id="div_fig3">

<img src="tmp_2302.02890/./KLFs.png" alt="Fig4" width="100%"/>

**Figure 4. -** Analysis of the de-reddened luminosity functions corresponding to the NSD outer (left panel) and inner (right panel) regions. The reduced $\chi^2$ of the fit is shown in each panel. The 14 theoretical models used are grouped into 5 broader age bins to decrease the degeneracy between models with similar ages. (*KLFs*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

192  publications in the last 7 days.
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers